In [ ]:
# 전체 데이터 들고오기 (influxdb 출력)
from influxdb_client import InfluxDBClient

# influxdb connect
url = 'http://133.186.251.19:8086'
token = 'auLmfVaJpvWUbnMxhbixfgq5JjFrleKTNxnFphRZ_tfVHoypyyXhBe3zHT07tqRTylE15VRmjuNFX9-u9uv6nA=='
org = 'TXT'
bucket = 'TxT-iot'

# influxdb bucket에서 data read
with InfluxDBClient(url=url, token=token, org=org) as client:
    query_api = client.query_api()

    tables = query_api.query('from(bucket: "TxT-iot") |> range(start: -1d)')

    for table in tables:
        for record in table.records:
            print(str(record["_time"]) + " - " + record.get_measurement()
                  + " " + record.get_field() + "=" + str(record.get_value()))


In [ ]:
#humidity
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime

# InfluxDB 클라이언트 인스턴스 생성
client = InfluxDBClient(url=url, token=token)
query_api = client.query_api()

# 쿼리 구성
query = '''
    from(bucket: "TxT-iot")
        |> range(start: -1h)
        |> filter(fn: (r) => r["_measurement"] == "humidity")
        |> filter(fn: (r) => r["_field"] == "value")
        |> timeShift(duration: 9h)
'''

# 쿼리 실행 및 결과 가져오기
result = client.query_api().query(org=org, query=query)

# 결과 출력
for table in result:
    for record in table.records:
        print(f"Time: {record.get_time()}, Measurement: {record.get_measurement()}, Value: {record.get_value()}")

In [ ]:
# 1분 간격으로 humidity의 가장 최근 데이터 가져오기
import time
from influxdb_client import InfluxDBClient

# InfluxDB 연결 정보 설정
url = 'http://133.186.251.19:8086'
token = 'auLmfVaJpvWUbnMxhbixfgq5JjFrleKTNxnFphRZ_tfVHoypyyXhBe3zHT07tqRTylE15VRmjuNFX9-u9uv6nA=='
org = 'TXT'
bucket = 'TxT-iot'

# InfluxDB 클라이언트 인스턴스 생성
client = InfluxDBClient(url=url, token=token)

def get_recent_humidity():
    # 쿼리 구성
    query = '''
        from(bucket: "TxT-iot")
            |> range(start: -1m)
            |> filter(fn: (r) => r["_measurement"] == "humidity")
            |> last()
            |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
            |> timeShift(duration: 9h)
    '''
    # 쿼리 실행 및 결과 가져오기
    result = client.query_api().query_data_frame(org=org, query=query)

    # 결과 출력
    if not result.empty:
        print(result['_time'][0])  # 가장 최근의 시간
        print(result['value'][0])  # 가장 최근의 값
    else:
        print("No data available.")

# 1분마다 get_recent_humidity 함수를 실행
while True:
    get_recent_humidity()
    time.sleep(60)  # 60초(1분) 대기

In [ ]:
#temperature
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime

# InfluxDB 클라이언트 인스턴스 생성
client = InfluxDBClient(url=url, token=token)
query_api = client.query_api()

# 쿼리 구성
query = '''
    from(bucket: "TxT-iot")
        |> range(start: -1h)
        |> filter(fn: (r) => r["_measurement"] == "temperature")
        |> filter(fn: (r) => r["_field"] == "value")
        |> timeShift(duration: 9h)
'''

# 쿼리 실행 및 결과 가져오기
result = client.query_api().query(org=org, query=query)

# 결과 출력
for table in result:
    for record in table.records:
        print(f"Time: {record.get_time()}, Measurement: {record.get_measurement()}, Value: {record.get_value()}")

In [ ]:
#1분에 1개씩만 출력하면 scheduleing 사용이 과할 수 있음 > 쿼리문으로 직접 temperature의 가장 최근 데이터 가져오기
import time
from influxdb_client import InfluxDBClient

# InfluxDB 연결 정보 설정
url = 'http://133.186.251.19:8086'
token = 'auLmfVaJpvWUbnMxhbixfgq5JjFrleKTNxnFphRZ_tfVHoypyyXhBe3zHT07tqRTylE15VRmjuNFX9-u9uv6nA=='
org = 'TXT'
bucket = 'TxT-iot'

# InfluxDB 클라이언트 인스턴스 생성
client = InfluxDBClient(url=url, token=token)

def get_recent_temperature():
    # 쿼리 구성
    query = '''
        from(bucket: "TxT-iot")
            |> range(start: -1m)
            |> filter(fn: (r) => r["_measurement"] == "temperature")
            |> last()
            |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
            |> timeShift(duration: 9h)
    '''
    # 쿼리 실행 및 결과 가져오기
    result = client.query_api().query_data_frame(org=org, query=query)

    # 결과 출력
    if not result.empty:
        print(result['_time'][0])  # 가장 최근의 시간
        print(result['value'][0])  # 가장 최근의 값
    else:
        print("No data available.")

# 1분마다 get_recent_temperature 함수를 실행
while True:
    get_recent_temperature()
    time.sleep(60)  # 60초(1분) 대기
